In [489]:
import numpy as np
import pandas as pd

In [490]:
df_ground=pd.read_csv(r"C:\Users\MATEJ\Documents\BDS SEMESTER 2\Projects in Data Science\Project\metadata.csv")
print('Number total:',len(df_ground))
print('Number of females:',len(df_ground[df_ground['gender']=='FEMALE']))
print('Number of males:',len(df_ground[df_ground['gender']=='MALE']))
print()
df_gender=df_ground[df_ground['gender'].notna()]
print('Number of data with gender data:',len(df_gender))
print('Number female cancer:',len(df_ground[df_ground['gender']=='FEMALE']))
print('Number male cancer:',len(df_ground[df_ground['gender']=='MALE']))
print()
print(df_gender['diagnostic'].value_counts())
df_ground

Number total: 2298
Number of females: 753
Number of males: 741

Number of data with gender data: 1494
Number female cancer: 753
Number male cancer: 741

diagnostic
BCC    845
ACK    283
SCC    192
NEV     75
MEL     52
SEK     47
Name: count, dtype: int64


,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,...,NaN,NEV,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,5.0,BCC,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,5.0,BCC,True,True,False,False,True,True,PAT_684_1302_588.png,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,PAT_1708,3156,NaN,NaN,NaN,NaN,73,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1708_3156_175.png,False
2294,PAT_46,880,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,12.0,BCC,True,True,False,True,False,False,PAT_46_880_140.png,True
2295,PAT_1343,1217,NaN,NaN,NaN,NaN,74,NaN,NaN,NaN,...,NaN,SEK,False,False,False,False,False,False,PAT_1343_1217_404.png,False
2296,PAT_326,690,False,False,POMERANIA,POMERANIA,58,True,FEMALE,True,...,4.0,BCC,True,False,False,False,False,True,PAT_326_690_823.png,True


In [ ]:
## New column with 1 for cancer, 0 for clear
df_ground['cancer']=np.where( (df_ground[ 'diagnostic']=='BCC') ^ (df_ground['diagnostic']=='MEL') ^ (df_ground['diagnostic']=='SCC'),1,0 )

In [785]:
## SETUP
import matplotlib.pyplot as plt
df=pd.DataFrame() ## empty df
filepath=r"C:\Users\MATEJ\Documents\BDS SEMESTER 2\Projects in Data Science\Project\data\data\img_0000" ## base path
start_im_id=200 ## INCLUDED
end_im_id=300 ## EXCLUDED

In [786]:
## MASK
from skimage.color import rgb2gray
from skimage.filters import gaussian, threshold_otsu
def get_mask(image):
    gray_im = rgb2gray(image)
    blurred_im = gaussian(gray_im, sigma=1.0)
    t=threshold_otsu(blurred_im)
    mask = gray_im<t
    return mask

In [787]:
## ASYMMETRY
from extract_features_examples import asymmetry#, find_midpoint_v1
for i in range(start_im_id,end_im_id):
    img=filepath[:-len(str(i))]+str(i)+'.png' ## modify path with current i
    im = (plt.imread(img)*255).astype(np.uint8)
    im=im[...,:3] ## REMOVE ALPHA

    mask=get_mask(im)
    
    #df.loc[i,'image_id']=i
    df.loc[i,'A - asymmetry']=asymmetry(mask)

In [788]:
## BORDER - Convexity and streaks
from extract_features_examples import convexity_score,measure_streaks
for i in range(start_im_id,end_im_id):
    img=filepath[:-len(str(i))]+str(i)+'.png' ## modify path with current i
    im = (plt.imread(img)*255).astype(np.uint8)
    im=im[...,:3] ## REMOVE ALPHA

    mask=get_mask(im)

    df.loc[i,'B - convexity']=convexity_score(mask)
    df.loc[i,'B - streaks']=measure_streaks(im)

In [789]:
## COLOR - Blue veil
from extract_features_examples import measure_blue_veil
for i in range(start_im_id,end_im_id):
    img=filepath[:-len(str(i))]+str(i)+'.png' ## modify path with current i
    im = (plt.imread(img)*255).astype(np.uint8)
    im=im[...,:3] ## REMOVE ALPHA

    df.loc[i,'C - blueveil']=measure_blue_veil(im)

In [790]:
## Combine current DF with cancer values - 1=cancer, 0=no cancer
for i in range(start_im_id,end_im_id):
    df.loc[i,'Cancer']=df_ground.loc[i,'cancer']

In [791]:
## Save it to a file so I don't have to run blueveil every time...
pd.DataFrame.to_csv(df,'features.csv',index_label='image_id')
df

,A - asymmetry,B - convexity,B - streaks,C - blueveil,Cancer
200,0.2041,0.701255,0.000000,62009.0,0.0
201,0.5151,0.147241,0.000000,110570.0,1.0
202,0.3311,0.713009,0.000000,144193.0,1.0
203,0.3465,0.093326,0.000000,102578.0,0.0
204,0.3007,0.515782,0.000000,110348.0,1.0
...,...,...,...,...,...
295,0.5041,0.358014,0.000000,109876.0,1.0
296,0.2570,0.571990,1.657068,74531.0,1.0
297,0.5121,0.140719,0.000000,8.0,0.0
298,0.3749,0.344104,0.000000,110220.0,1.0


In [802]:
## Read and drop the index
df=pd.read_csv(r"C:\Users\MATEJ\Documents\BDS SEMESTER 2\Projects in Data Science\Project\features.csv",index_col=0)
df

,A - asymmetry,B - convexity,B - streaks,C - blueveil,Cancer
image_id,,,,,
200,0.2041,0.701255,0.000000,62009.0,0.0
201,0.5151,0.147241,0.000000,110570.0,1.0
202,0.3311,0.713009,0.000000,144193.0,1.0
203,0.3465,0.093326,0.000000,102578.0,0.0
204,0.3007,0.515782,0.000000,110348.0,1.0
...,...,...,...,...,...
295,0.5041,0.358014,0.000000,109876.0,1.0
296,0.2570,0.571990,1.657068,74531.0,1.0
297,0.5121,0.140719,0.000000,8.0,0.0


In [803]:
## Training and testing data
from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(5,shuffle=True)

## Split into features and cancer values
X=df.drop(columns='Cancer')
y=df['Cancer']

## Split based on cancer values
for train_idx, test_idx in skf.split(X, y):
    train_df = df.iloc[train_idx]  # 80%
    test_df = df.iloc[test_idx]    # 20%
    break
## Gives training data (80%) and testing data (20%)

In [804]:
## Training and validation
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4)

## Split into features and cancer values
X = train_df.drop(columns='Cancer')
y = train_df['Cancer']

## Split based on cancer values
for train_idx, test_idx in sss.split(X, y):
    train_df = df.iloc[train_idx]  # 60%
    valid_df = df.iloc[test_idx]    # 40%
## Takes previous training data
## Gives training data (60%) and validation data (40%)

In [805]:
## Remove 'Cancer' for normalizing features
train_df_cancer=train_df.copy()
train_df=train_df.drop(columns='Cancer')

valid_df_cancer=valid_df.copy()
valid_df=valid_df.drop(columns='Cancer')

In [806]:
## Normalize the features
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(train_df)

#Apply to data to both train and test test
train_scaled = scaler.transform(train_df)
valid_scaled = scaler.transform(valid_df)

train_df_scaled = pd.DataFrame(train_scaled,columns=['A - asymmetry','B - convexity', 'B - streaks', 'C - blueveil'],dtype=np.float64)
valid_df_scaled= pd.DataFrame(valid_scaled,columns=['A - asymmetry','B - convexity', 'B - streaks', 'C - blueveil'],dtype=np.float64)

In [807]:
## Train a classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knntrained = knn.fit(train_df_scaled, train_df_cancer['Cancer']) ## Training data and cancer values

In [808]:
## TEMP until we find a better way of finding accuracy
from sklearn.metrics import accuracy_score
predited=knntrained.predict(valid_df_scaled)
acc_knn = accuracy_score(valid_df_cancer['Cancer'], predited)
acc_knn

0.5625